In [83]:
import os, sys
from PIL import Image
import glob
import cv2 as cv
from dotenv import load_dotenv
import numpy as np
from matplotlib import pyplot as plt
import openai
import logging as log
import time
from io import BytesIO
from goa import save_images
import json
import base64

load_dotenv()

True

In [84]:
def get_file_name(path: str):
    return path.split("/")[-1]

In [85]:
def fill_image(image, text_prompt, n=4, format="b64_json"):

    try:
        log.info("Generating images...")
        start_t = time.perf_counter()

        response = openai.Image.create_edit(
            image=image,
            prompt=text_prompt,
            n=n,
            size="1024x1024",
            response_format=format,
        )

        duration = time.perf_counter() - start_t
        log.info(f"Generating images took {duration:.2f}s")

        return response, "Images generated successfully!"

    except openai.error.OpenAIError as e:
        log.info(e.http_status)
        log.info(e.error)
        log.info(e)
        return None, str(e)

In [86]:
def get_image(response):
    image_data = response['data'][0]["b64_json"]

    image = Image.open(BytesIO(base64.b64decode(image_data)))
    return image

In [87]:
def fill_watermarks():
    to_fill = glob.glob(f"{os.getenv('NO_WM_IMG_MNT')}/*png")
    for img_path in to_fill:

        print(f"Starting with {img_path}")
        # Read the image file from disk and resize it
        img = Image.open(img_path)
        w, h = img.size
        box = (w-1024, h-1024, w, h)
        crop = img.crop(box)

        # Convert the image to a BytesIO object
        byte_stream = BytesIO()
        crop.save(byte_stream, format='PNG')
        byte_array = byte_stream.getvalue()
        prompt = get_file_name(img_path)
        r, msg = fill_image(byte_array, prompt, n=1,)
        filling = get_image(r)
        img.paste(filling, box=box)

        target_path = f"{os.getenv('TO_UPSCALE_MNT')}/{get_file_name(img_path)}"
        print(f"Saving to: {target_path}")
        img.save(target_path)

In [88]:
fill_watermarks()

Starting with mnt/images/3_no_watermark/a modernist poster showing a greek vase showing a pregnant woman playing with a faun.png
Type of response <class 'openai.openai_object.OpenAIObject'>
Saving to: mnt/images/4_to_upscale/a modernist poster showing a greek vase showing a pregnant woman playing with a faun.png
Starting with mnt/images/3_no_watermark/a modest poster of abstract floris shapes. use the style of medieval art. use soft colours.png
Type of response <class 'openai.openai_object.OpenAIObject'>
Saving to: mnt/images/4_to_upscale/a modest poster of abstract floris shapes. use the style of medieval art. use soft colours.png
Starting with mnt/images/3_no_watermark/a poster of biological abstract shapes. mix the styles of Stanislaw Wyspiański and  USSR.png
Type of response <class 'openai.openai_object.OpenAIObject'>
Saving to: mnt/images/4_to_upscale/a poster of biological abstract shapes. mix the styles of Stanislaw Wyspiański and  USSR.png
Starting with mnt/images/3_no_watermar